In [1]:
import numpy as np
import cv2
import imutils

NMS_THRESHOLD = 0.3
MIN_CONFIDENCE = 0.2

MAX_PEDESTRIANS = 4

In [2]:

def pedestrian_detection(image, model, layer_name, personidz=0):
    (H, W) = image.shape[:2]
    results = []

    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    model.setInput(blob)
    layerOutputs = model.forward(layer_name)

    boxes = []
    centroids = []
    confidences = []

    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            if classID == personidz and confidence > MIN_CONFIDENCE:
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))

                boxes.append([x, y, int(width), int(height)])
                centroids.append((centerX, centerY))
                confidences.append(float(confidence))

    idzs = cv2.dnn.NMSBoxes(boxes, confidences, MIN_CONFIDENCE, NMS_THRESHOLD)

    if len(idzs) > 0:
        for i in idzs.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            res = (confidences[i], (x, y, x + w, y + h), centroids[i])
            results.append(res)

    return results


In [3]:

labelsPath = "coco.names"
LABELS = open(labelsPath).read().strip().split("\n")

weights_path = "yolov4-tiny.weights"
config_path = "yolov4-tiny.cfg"

model = cv2.dnn.readNetFromDarknet(config_path, weights_path)

layer_name = model.getLayerNames()
layer_name = [layer_name[i - 1] for i in model.getUnconnectedOutLayers()]

# Access the webcam with (0) or else whatever video wanna put using "video1.mp4"
#cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture("./push.mp4")

while True:
    (grabbed, image) = cap.read()
    ret, frames = cap.read()
    frames = cv2.resize(frames, (0, 0), fx=0.5, fy=0.5)

    if not grabbed:
        break

    # Resize the frame for better processing speed
    image = imutils.resize(image, width=700)

    # Detect pedestrians in the frame
    results = pedestrian_detection(image, model, layer_name, personidz=LABELS.index("person"))

    # Reset the pedestrian count for each frame
    pedestrian_count = 0

    for res in results:
        cv2.rectangle(image, (res[1][0], res[1][1]), (res[1][2], res[1][3]), (0, 255, 0), 2)
        pedestrian_count += 1

    # Check if the pedestrian count exceeds the threshold
    if pedestrian_count > MAX_PEDESTRIANS:
        # Raise an alert or trigger an alarm here
        print("Pedestrian overcrowding detected! Alert or alarm triggered.")

    # Display the frame with pedestrian detection
    cv2.imshow("Pedestrian Detection", image)

    key = cv2.waitKey(1)
    if key == 27:  # Press 'Esc' to exit the loop
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()



Pedestrian overcrowding detected! Alert or alarm triggered.
Pedestrian overcrowding detected! Alert or alarm triggered.
Pedestrian overcrowding detected! Alert or alarm triggered.
Pedestrian overcrowding detected! Alert or alarm triggered.
Pedestrian overcrowding detected! Alert or alarm triggered.
Pedestrian overcrowding detected! Alert or alarm triggered.
Pedestrian overcrowding detected! Alert or alarm triggered.
Pedestrian overcrowding detected! Alert or alarm triggered.
Pedestrian overcrowding detected! Alert or alarm triggered.
Pedestrian overcrowding detected! Alert or alarm triggered.
Pedestrian overcrowding detected! Alert or alarm triggered.
Pedestrian overcrowding detected! Alert or alarm triggered.
Pedestrian overcrowding detected! Alert or alarm triggered.
Pedestrian overcrowding detected! Alert or alarm triggered.
Pedestrian overcrowding detected! Alert or alarm triggered.
Pedestrian overcrowding detected! Alert or alarm triggered.
Pedestrian overcrowding detected! Alert 

error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


: 